# PSB injection bump in Xsuite - basics

In [1]:
%matplotlib notebook

In [2]:
# Import modules
import matplotlib.pyplot as plt
import numpy as np

import xtrack as xt
import xpart as xp
import xdeps as xd

from cpymad.madx import Madx

## Get MAD-X model

In [3]:
# Load mad thick model and apply element shifts
mad = Madx()
mad.input('''
    call, file = 'psb.seq';
    call, file = 'psb_fb_lhc.str';

    beam, particle=PROTON, pc=0.5708301551893517;
    use, sequence=psb1;
''')

# Add shifts on injection bumpers
mad.input('''
    select,flag=error,pattern=bi1.bsw1l1.1*;
    ealign, dx=-0.0057;

    select,flag=error,clear;
    select,flag=error,pattern=bi1.bsw1l1.2*;
    select,flag=error,pattern=bi1.bsw1l1.3*;
    select,flag=error,pattern=bi1.bsw1l1.4*;
    ealign, dx=-0.0442;
''')

mad.twiss() # To check that the model is fine



  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.00  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2023.05.05               +
  + Execution date: 2023.11.24 09:09:21      +
  ++++++++++++++++++++++++++++++++++++++++++++




  /************************************************************************************

  *

  * PS Booster version  (draft) LS2 in MAD X SEQUENCE format

  * Generated the 28-AUG-2020 16:25:27 from Layout

  *

  ************************************************************************************/







/************************************************************************************/

/*                       TYPES DEFINITION                                           */

/************************************************************************************/



//---------------------- COLLIMATOR     ---------------------------------------------

BR_SPSCRAP     : COLLIMATOR  , L := 0;         ! PSB

<Table 'twiss': {name, keyword, s, betx, alfx, mux, bety, alfy, muy, x, px, y, py, t, pt, dx, dpx, dy, dpy, wx, phix, dmux, wy, phiy, dmuy, ddx, ddpx, ddy, ddpy, r11, r12, r21, r22, energy, l, angle, k0l, k0sl, k1l, k1sl, k2l, k2sl, k3l, k3sl, k4l, k4sl, k5l, k5sl, k6l, k6sl, k7l, k7sl, k8l, k8sl, k9l, k9sl, k10l, k10sl, k11l, k11sl, k12l, k12sl, k13l, k13sl, k14l, k14sl, k15l, k15sl, k16l, k16sl, k17l, k17sl, k18l, k18sl, k19l, k19sl, k20l, k20sl, ksi, hkick, vkick, tilt, e1, e2, h1, h2, hgap, fint, fintx, volt, lag, freq, harmon, slot_id, assembly_id, mech_sep, v_pos, bbcharge, xma, yma, sigx, sigy, lrad, parent, comments, re11, re12, re13, re14, re15, re16, re21, re22, re23, re24, re25, re26, re31, re32, re33, re34, re35, re36, re41, re42, re43, re44, re45, re46, re51, re52, re53, re54, re55, re56, re61, re62, re63, re64, re65, re66, kmax, kmin, calib, polarity, alfa, beta11, beta12, beta13, beta21, beta22, beta23, beta31, beta32, beta33, alfa11, alfa12, alfa13, alfa21, alfa22, alfa

## Build Xsuite line from MAD-X model

In [4]:
# Build line (thick elements)
line = xt.Line.from_madx_sequence(mad.sequence.psb1,
                                  allow_thick=True,
                                  enable_align_errors=True,
                                  deferred_expressions=True)

# Associate reference particle to the line
line.particle_ref = xp.Particles(mass0=xp.PROTON_MASS_EV, gamma0=mad.sequence.psb1.beam.gamma)

# Switch to exact model for bends (PTC-like, approppriate for small rings)
line.configure_bend_model(core='full', edge='full')

# Build tracker (by default single-core CPU)
line.build_tracker()

Converting sequence "psb1": 100%|█| 528/528 [00:00<00:00, 98

Found suitable prebuilt kernel `default_only_xtrack`.


### Inspect the line elements

In [5]:
line_table = line.get_table() 

In [6]:
line_table.rows[:12] # we can see that the injection bumpers are exploded into multiple simpler elements
                     # belonging to the same compound.

Table: 12 rows, 5 cols
name                           s element_type isthick compound_name    
psb1$start                     0 Marker         False                  
p01ring1$start                 0 Marker         False                  
drift_0                        0 Drift           True                  
bi1.bsw1l1.1_entry        0.1795 Marker         False bi1.bsw1l1.1     
bi1.bsw1l1.1_offset_entry 0.1795 XYShift        False bi1.bsw1l1.1     
bi1.bsw1l1.1_den          0.1795 DipoleEdge     False bi1.bsw1l1.1     
bi1.bsw1l1.1              0.1795 Bend            True bi1.bsw1l1.1     
bi1.bsw1l1.1_dex          0.4925 DipoleEdge     False bi1.bsw1l1.1     
bi1.bsw1l1.1_offset_exit  0.4925 XYShift        False bi1.bsw1l1.1     
bi1.bsw1l1.1_exit         0.4925 Marker         False bi1.bsw1l1.1     
drift_1                   0.4925 Drift           True                  
bi1.bsw1l1.2_entry        0.8695 Marker         False bi1.bsw1l1.2     

### Twiss defaults
By default the Xsuite twiss is 6d (gives error if RF is not present).
We can set different defaults for our line, for example switch to 4d twiss, and hide group "compound elements".

In [7]:
line.twiss_default['method'] = '4d'
line.twiss_default['group_compound_elements'] = True

###  Save reload
At any moment the line can be saved to a json file and reloaded

In [8]:
# Save
line.to_json('psb_00_from_mad.json')

In [9]:
# Reload
line = xt.Line.from_json('psb_00_from_mad.json')
line.build_tracker()

Loading line from dict: 100%|█| 1224/1224 [00:00<00:00, 2215

Done loading line from dict.           
Found suitable prebuilt kernel `default_only_xtrack`.


## Working with the line (basics)

### Twiss

Use the twiss method for analyzing the line model and diagnosing issues:

In [ ]:
tw = line.twiss()

Access scalar quantities (tunes, chromaticities, momentum compaction, rev period). For example: 

In [ ]:
tw.qx, tw.qy, tw.dqy, tw.c_minus, tw.T_rev0

Plot beta functions, orbit, dispersion. 

In [ ]:
fig = plt.figure(1, figsize=(6, 4*1.4))
sp1 = plt.subplot(3,1,1)
sp2 = plt.subplot(3,1,2, sharex=sp1)
sp3 = plt.subplot(3,1,3, sharex=sp1)

sp1.plot(tw.s, tw.betx)
sp1.plot(tw.s, tw.bety)
sp1.set_ylabel(r'$\beta_{x,y}$ [m]')
sp2.plot(tw.s, tw.dx)
sp2.plot(tw.s, tw.dy)
sp2.set_ylabel(r'$D_{x,y}$ [m]')
sp3.plot(tw.s, tw.x)
sp3.plot(tw.s, tw.dy)
sp3.set_ylim(-1e-3, 1e-3)
sp3.set_ylabel(r'x,y [m]')

### Working with twiss tables and inspecting line elements

Twiss table support several indexing modes. Here some examples:

In [ ]:
# Select rows and colums (rows selected by index)
tw.rows[20:25].cols['s', 'betx', 'bety']

In [ ]:
# Select rows and colums (rows selected by name)
tw.rows['br.spscrap11_entry':'br1.bpm1l3_entry'].cols['s', 'betx', 'bety']

In [ ]:
# Select rows and colums (rows selected by `s` range)
tw.rows[150.:153.:'s'].cols['s', 'betx', 'bety']

In [ ]:
# Select rows and colums (rows selected with regular expression)
tw.rows['.*bsw1l.*'].cols['s', 'betx', 'bety']

Line elements can be inspected with:

In [ ]:
line['br.bhz21']

### Deferred expressions imported from MAD-X

MAD-X deferred expressions are imported as variables attached to the line. For example:

In [ ]:
line.vars['k0bi1bsw1l11']._info()

We can set/read variables as follows:

In [ ]:
line.vars['k0bi1bsw1l11'] = 0.1

In [ ]:
print(line['bi1.bsw1l1.1'].k0)
print(line['bi1.bsw1l1.1_den'].r43)

In [ ]:
line.vars['k0bi1bsw1l11'] = 0.

In [ ]:
print(line['bi1.bsw1l1.1'].k0)
print(line['bi1.bsw1l1.1_den'].r43)

Knobs attached to the elements can be inspected or modified through `line.element_refs`. For example

In [ ]:
line.element_refs['bi1.bsw1l1.1'].k0._info()

### Match parameters

Variables can be used to optimize settings as to obtain certain targets in the twiss results (match). For example we can match the tunes:

In [ ]:
info = line.match(
    targets=[
        xt.Target('qx',  4.2, tol=1e-6),
        xt.Target('qy',  4.3, tol=1e-6)],
    vary=[
        xt.Vary('kbrqf', step=1e-5),
        xt.Vary('kbrqd', step=1e-5)],
)

In [ ]:
# Check
tw = line.twiss()
tw.qx, tw.qy

---------